### EGOR ZAITSEV - 07 452 541

### XIAOXU YAO - 07 498 897

In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.optimize import fmin, minimize

## Exercise 1: Consumption Savings Problem

Consider the following life-cycle Problem

$$\begin{aligned}&\max_{\{c_t\}_{t=0}^{T}}\sum_{t=0}^{T}\beta^tu(c_t)\\\text{s.t.}&u(c_t)=\frac{c_t^{1-\theta}-1}{1-\theta}\\&c_t+a_{t+1}=a_t(1+r)+w_t\\&a_0\text{ given and }a_{T+1}\ge0\end{aligned}$$

### Characterize the analytical solution for any $\beta,r,w_t$

#### Euler equation
Start off by writing out the objective up to the second element

$$\max_{\{c_t\}_{t=0}^{T}}u(c_0)+\beta u(c_1)+\dots$$

At the optimum, for any two arbitrary periods it must hold that

$$\frac{\partial u(\cdot)}{\partial c_0}=\beta\frac{\partial u(\cdot)}{\partial c_1}\frac{\partial c_1}{\partial c_0}$$

Using the budget constraint we may find that $\frac{\partial c_1}{\partial c_0}=1+r$ hence the Euler equation for period $t$ reads

$$c_t^{-\theta}=\beta(1+r)c_{t+1}^{-\theta}\iff\frac{c_{t+1}}{c_t}=[\beta(1+r)]^{\frac{1}{\theta}}$$

To express consumption explicitly we must solve the Bellman equation via backwardation

$$\nu_{T}(a_{T})=\max_{a_{T+1}}\biggr\{\frac{(w_T+(1+r)a_T-a_{T+1})^{1-\theta}-1}{1-\theta}+\nu_{T+1}(a_{T+1})\biggr\}$$

Which is obviously maximized if $a_{T+1}=0$ which leads us to $\nu_T(a_T)=\frac{(w_T+(1+r)a_T)^{1-\theta}-1}{1-\theta}$. Iterating backwards leads to an explicit form of the policy function $a_{T-t+1}(a_{T-t}$. Using this and the budget constraint we may solve for the consumption function $c_{t}$

#### $\beta$-dependence
As $\beta$ increases, the consumption path is skewed towards the early life which means the consumer consumes more income early. Equivalently, lower values of $\beta$ imply lower consumption levels early in life and higher consumption levels later in life. $\beta$ parameter is thus a time preference and a tradeoff between wealth allocation and wealth spending

#### $r$-dependence
Higher interest rates imply higher returns on capital. Sufficiently high interest rates make consumption path skew towards end life

#### $w_t$-dependence
Higher levels of income imply higher levels of consumption, and vice versa

### Compute the numerical solutions setting $w_0=10, a_0=0$ and $w_t=0$ for $t\ge1, \beta=0.99,r=0.05$ and any $\theta$

In [74]:
# PARAMETERS
theta = 3
beta = 0.99
r = 0.05

# LIFE LENGTH
T = 10

# ENDOWMENT
w = [0.0] * T
w[0] = 10.
w = np.array(w)

# SAVINGS
a = [5] * (T-1)
a = a + [0]
a = np.array(a)

In [69]:
def crra(c, theta, tol=1e-5):
    """
    CRRA utility function (constant risk aversion).

    Arguments:
        c - consumption vector
        theta - risk aversion parameter
        tol - death tolerance

    Returns:
        function value
    """
    if theta != 0:
        if c < tol:
            u = -tol**(-1)
        else:
            u = (c**(1-theta) - 1)/(1-theta)
    else:
        return 'theta cannot be zero'
    
    return u


In [70]:
def budgetConstraint(a, w, r, T):
    """
    Budget constraint solved for consumption.

    Arguments:
        a - savings vector
        w - income vector
        r - interest rate
        T - agent's life length

    Returns:
        consumption vector assuming the constraint is binding
    """

    c = [0] * (T)
    c[0] = w[0] - a[0]
    for t in range(1, T-1):
        c[t] = w[t] + (1+r)*a[t-1] - a[t]
    c[T-1] = w[T-1] + (1+r)*a[T-2]

    return np.array(c)


In [71]:
def bellman(a, w, r, T, budgetConstraint, u, theta, beta):
    """
    Backward induction on the Bellman equation. Finite-horizon problems are solved via backwardation.

    Arguments:
        a - savings vector
        w - income vector
        r - interest rate
        T - agent's life length
        budgetConstraint - transform function
        u - utility function
        theta - relative risk aversion parameter
        beta - time preference parameter
    Returns:
        function value
    """
    c = budgetConstraint(a, w, r, T)
    v = u(c[-1], theta)

    for period in reversed(range(len(c) - 1)):
        v = u(c[period], theta) + beta*v
    
    return -v # return negative Bellman because we use minimizers

In [75]:
a1 = fmin(bellman, x0=a, args=(w, r, T, budgetConstraint, crra, theta, beta), maxiter=10e4)
c1 = budgetConstraint(a1, w, r, T)

Optimization terminated successfully.
         Current function value: -1.650965
         Iterations: 1417
         Function evaluations: 2022


In [76]:
data = {'w': w, 'a0': a, 'a*': a1, 'c*': c1}
df = pd.DataFrame(data)
print('numerical solution for T=10 and theta=3\n{}'.format(df.round(2)))

numerical solution for T=10 and theta=3
      w  a0    a*    c*
0  10.0   5  8.83  1.17
1   0.0   5  8.09  1.18
2   0.0   5  7.29  1.20
3   0.0   5  6.44  1.22
4   0.0   5  5.53  1.23
5   0.0   5  4.56  1.25
6   0.0   5  3.53  1.26
7   0.0   5  2.43  1.28
8   0.0   5  1.25  1.30
9   0.0   0 -0.06  1.31


To solve the Bellman equation we use simplex method (aka Nelder-Mead algorithm) which helps us minimize the objective without specifying neither first nor second derivative of the objective

Note that the vectors $\mathbf a_0$ and $\mathbf a^*$ are shifted in time by one increment. Using such a structure for the budget constraint we impose $a_0=0$ and $a_{T+1}\ge0$. We see that $a^*_{T+1}$ is slightly below zero which is a sign of truncation error of this numerical method

### How do you deal with the constraint $c_t>0$ for all $t$?

We introduce 'death tolerance' which is a number slightly above zero ($10e-5$, to be precise). If the algorithm decides to choose a consumption level below the tolerance we 'penalize' it by assigning a very negative value to consumption utility ($10e5$, an inverse of death penalty). Hence, an algorithm never chooses consumption levels close to zero

## Exercise 2: The Consumption-Savings Problem with Human Capital

Consider a two period $t=\{1,2\}$ consumption-savings problem with human capital

 - $c_t,k_t,h_t$ are consumption, stock of physical and human capital

 - $i_k,i_h$ are investment in physical and human capital

 - $\delta_k,\delta_h\in[0,1]$ are depreciation rates for physical and human capital

 - $k_2=(1-\delta_k)k_1+i_k$ and $h_2=[(1-\delta_h)h_1+i_h]^\mu$

 - $k_2\ge0$ is a borrowing constraint on physical capital

- $i_h\ge0$ is an irreversibiility constraint on human capital

- $r_k,r_h$ are returns on physical and human capital

- $\gamma,\beta,\mu$ are risk aversion, discount factor and decreasing marginal returns to human capital investment

where optimization problem is given by

$$\begin{aligned}&\max_{c_1,c_2,i_k,i_h}\biggr\{\frac{c_1^{1-\gamma}-1}{1-\gamma}+\beta\frac{c_2^{1-\gamma}-1}{1-\gamma}\biggr\}\\\text{s.t.}&(1-\delta_k)k_1+i_k\ge0\\&i_h\ge0\\&c_1+i_k+i_h-(r_kk_1+r_hh_1)=0\\&c_2-\{(1+r_k)[(1-\delta_k)k_1+i_k]+r_h[(1-\delta_h)h_1+i_h]^\mu\}=0\end{aligned}$$

In [97]:
gamma, beta, rk, rh, mu, dk, dh = 2.0, 0.96, 0.1, 1.4, 0.8, 0.05, 0.05

### Derive the Kuhn-Tucker conditions of this optimization problem

Note first that

$$\begin{aligned}c_1&=r_kk_1+r_hh_1-i_k-i_h\\c_2&=(1+r_k)[(1-\delta_k)k_1+i_k]-r_h[(1-\delta_h)h_1+i_h]^\mu\end{aligned}$$

and write out the Lagrangian $\mathcal L(\cdot)$

$$\mathcal L(\cdot)=\frac{c_1^{1-\gamma}-1}{1-\gamma}+\beta\frac{c_2^{1-\gamma}-1}{1-\gamma}+\lambda_1[(1-\delta_k)k_1+i_k]+\lambda_2i_h\to\max_{\circ}$$

KKT conditions are then given by

$$\frac{\partial \mathcal L(\cdot)}{\partial c_t}=0$$

$$i_h\ge0$$

$$(1-\delta_k)k_1+i_k\ge0$$

$$\lambda_1,\lambda_2\ge0$$

$$\lambda_1[(1-\delta_k)k_1+i_k]=0$$

$$\lambda_2i_h=0$$

$$t\in\{1,2\}$$

### Use routine of your choice to solve the above problem for the following initial endowments

- $k_1=1,h_1=5$

- $k_1=1,h_1=1$

- $k_1=1,h_1=0.2$

In [197]:
kh1 = [1, 5]
kh2 = [1, 1]
kh3 = [1, 0.2]

In [190]:
def crra(c, gamma=gamma):
    """
    CRRA utility function.

    Arguments:
        c - consumption input
        gamma - risk aversion parameter

    Returns:
        function value
    """
    if gamma != 0:
        u = (c**(1-gamma) - 1)/(1-gamma)
    else:
        return 'theta cannot be zero'
    
    return (-1)*u


In [231]:
def objective(kh, gamma=gamma, beta=beta, rk=rk, rh=rh):
    """
    Objective function to maximize.

    Arguments:
        kh - physical and human input array
        gamma - risk aversion parameter
        beta - discount factor
        rk, rh, ik, ih - self explanatory

    Returns:
        function value
    """
    k1, h1, ik, ih = khi
    
    c1 = rk*k1 + rh*h1 - ik - ih # function of consumption in period 1
    c2 = (1+rk)*((1-dk)*k1 + ik) - rh*((1-dh)*h1 + ih)**mu # function of consumption in period 2

    return crra(c1) + beta*crra(c2)

In [236]:
def c1(kh, rk=rk, rh=rh, ik=ik, ih=ih):
    k1, h1, ik, ih = khi
    return rk*k1 + rh*h1 - ik - ih

def c2(khi, rk=rk, rh=rh, dk=dk, dh=dh, mu=mu, ik=ik, ih=ih):
    k1, h1, ik, ih = khi
    return (1+rk)*((1-dk)*k1 + ik) - rh*((1-dh)*h1 + ih)**mu

In [237]:
def conik(kh):
    k1, _, ik, _ = khi
    
    return (1-dk)*k1 + ik

def conih(kh):
    _, _, _, ih = khi

    return ih


In [238]:
con1 = {'type': 'eq', 'fun': c1}
con2 = {'type': 'eq', 'fun': c2}
con3 = {'type': 'ineq', 'fun': conik}
con4 = {'type': 'ineq', 'fun': conih}
cons = [con1, con2, con3, con4]

sol = minimize(objective, khi, method='SLSQP', constraints=cons)

In [235]:
sol

     fun: -1.9595181751711066
     jac: array([-1.49011612e-08, -2.68220901e-07,  2.08616257e-07,  2.08616257e-07])
 message: 'Optimization terminated successfully'
    nfev: 188
     nit: 33
    njev: 33
  status: 0
 success: True
       x: array([17128.48390918, 16491.08368988,  7088.15165073, 15495.21476456])